In [ ]:
import os
import time
import threading
from collections import deque
from enum import Enum

import cv2
import numpy as np
import pandas as pd
import mediapipe as mp

import torch
import torch.nn.functional as F

from ultralytics import YOLO

from local_landmark import LocalLandmark
from realsense_camera import RealSenseCamera
from yolo_object import YoloObject
from hand_helper import HandHelper
from object_tracker import ObjectTracker
from robot_controller import RobotController

In [ ]:
SAMPLING_FPS = 10
SAMPLING_SECONDS = 1
PREDICTION_WINDOW_SIZE = 5
sequence_length = int(SAMPLING_SECONDS*SAMPLING_FPS)
frame_interval = 1.0/SAMPLING_FPS

IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480

HAND_CONNECTIONS = ((0, 1), (0, 5), (9, 13), (13, 17), (5, 9), (0, 17), (1, 2), (2, 3), (3, 4), (5, 6), (6, 7), (7, 8),
                    (9, 10), (10, 11), (11, 12), (13, 14), (14, 15), (15, 16), (17, 18), (18, 19), (19, 20))

class Action(Enum):
    IDLE = 0
    PICK = 1
    PLACE = 2
    SCREW_WRENCH = 3

OBJECT_NAMES = {
    0: "small_screw",
    1: "big_screw",
    2: "small_wrench",
    3: "big_wrench",
    4: "cap",
    5: "barrel",
    6: "piston",
    7: "support",
    8: "air_connector",
    9: "nut"
}

num_classes_actions = len(Action)
num_classes_objects = len(OBJECT_NAMES) + 1 # Last class for empty hand

label_colors = [np.random.random(3)*255 for _ in range(len(OBJECT_NAMES))]

In [ ]:
mp_hands = mp.solutions.hands

yolo_model_path = os.path.join("weights", "yolov9c_fine_tuned.pt")
yolo_model = YOLO(yolo_model_path)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def get_landmarks_from_flattened_array(flattened_landmarks):
    N_RIGHT_HAND_LANDMARKS = 21
    N_LEFT_HAND_LANDMARKS = 21

    right_hand_landmarks = []
    left_hand_landmarks = []

    cursor = 0

    for _ in range(N_RIGHT_HAND_LANDMARKS):
        cursor_end_position = cursor + 4
        lm_sub_array = flattened_landmarks[cursor:cursor_end_position]
        right_hand_landmarks.append(LocalLandmark.from_np_array(lm_sub_array))
        cursor = cursor_end_position

    for _ in range(N_LEFT_HAND_LANDMARKS):
        cursor_end_position = cursor + 4
        lm_sub_array = flattened_landmarks[cursor:cursor_end_position]
        left_hand_landmarks.append(LocalLandmark.from_np_array(lm_sub_array))
        cursor = cursor_end_position

    return right_hand_landmarks, left_hand_landmarks

def one_hot_encode_class(class_index, num_classes):
    one_hot_vector = np.zeros(num_classes, dtype=int)
    one_hot_vector[class_index] = 1
    return one_hot_vector

def get_selected_frame_data(frame_landmarks_data, frame_objects_data):
    selected_frame_data = []

    right_hand_landmarks, left_hand_landmarks = get_landmarks_from_flattened_array(frame_landmarks_data)
    for i in range(2):
        if i == 0:
            hand_landmarks = right_hand_landmarks
        else:
            continue # Select only the right hand
            hand_landmarks = left_hand_landmarks
            
        wrist_landmark = hand_landmarks[0]
        for lm in hand_landmarks:
            # Absolute coordinates (with respect to camera frame)
            selected_frame_data.append(lm.x)
            selected_frame_data.append(lm.y)
            # selected_frame_data.append(lm.d/IMAGE_WIDTH)

            # Relative coordinates (relative to the wrist landmark)
            selected_frame_data.append(lm.x - wrist_landmark.x)
            selected_frame_data.append(lm.y - wrist_landmark.y)
            selected_frame_data.append(lm.z)

    if frame_objects_data is not None:
        right_hand_object_found = False
        for frame_object_data in frame_objects_data:
            label_id, in_hand, x1, y1, x2, y2 = frame_object_data
            if in_hand == 0: # Only objects held in the right hand are considered
                selected_frame_data = np.concatenate((selected_frame_data, one_hot_encode_class(label_id, num_classes_objects)), axis=0)
                right_hand_object_found = True
                break
        
        if not right_hand_object_found:
            selected_frame_data = np.concatenate((selected_frame_data, one_hot_encode_class(num_classes_objects - 1, num_classes_objects)), axis=0)

    return selected_frame_data

In [ ]:
def process_landmarks(frame, model):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame.flags.writeable = False
    results = model.process(frame)
    return results

def extract_hand_landmarks(hand_index, multi_hand_landmarks, multi_handedness, extract_depth=False, camera=None):
    """hand_index: 0 = right, 1 = left"""
    hand_landmarks_list = []
    hand_index_in_multi_landmarks = -1
    if multi_hand_landmarks and multi_handedness:
        for possible_hand_index in range(len(multi_hand_landmarks)):
            handedness_classification = multi_handedness[possible_hand_index].classification[0]
            handedness_index = handedness_classification.index
            handedness_score = handedness_classification.score
            if handedness_index == hand_index: # and handedness_score > 0.7
                hand_index_in_multi_landmarks = possible_hand_index
                break

    for i in range(21):
        if hand_index_in_multi_landmarks >= 0:
            lm = LocalLandmark.from_mediapipe_hand_landmark(multi_hand_landmarks[hand_index_in_multi_landmarks].landmark[i])
            if extract_depth:
                lm.set_depth(camera.get_depth(int(lm.x*IMAGE_WIDTH), int(lm.y*IMAGE_HEIGHT)))
            hand_landmarks_list.append(lm)
        else:
            hand_landmarks_list.append(LocalLandmark(0, 0, 0, 0))
                
    return hand_landmarks_list

def draw_landmarks(frame, landmarks, side, connections):
    """side: 0 = right, 1 = left"""

    if all([lm.is_empty() for lm in landmarks]):
        return

    right_landmarks_color = (0, 0, 255)
    left_landmarks_color = (255, 0, 0)

    for lm in landmarks:
        if side == 0:
            landmarks_color = right_landmarks_color
        elif side == 1:
            landmarks_color = left_landmarks_color
        else:
            return
        cv2.circle(frame, (int(lm.x*IMAGE_WIDTH), int(lm.y*IMAGE_HEIGHT)), 2, landmarks_color, 2)

    for connection in connections:
        if connection[1] < len(landmarks):
            start_point = (int(landmarks[connection[0]].x*IMAGE_WIDTH), int(landmarks[connection[0]].y*IMAGE_HEIGHT))
            end_point = (int(landmarks[connection[1]].x*IMAGE_WIDTH), int(landmarks[connection[1]].y*IMAGE_HEIGHT))
            cv2.line(frame, start_point, end_point, (255, 255, 255), 2)

In [ ]:
def show_probability_bars(frame, probabilities):
    bar_colors = ((255, 0 , 0), (0, 255, 0), (0, 0, 255), (255, 255, 0))
    for i, action_prob in enumerate(probabilities):
        cv2.rectangle(frame, (0, 35 + i*20), (int(action_prob*100), 55 + i*20), bar_colors[i], -1)
        label_color = (0, 255, 255) if action_prob >= 0.9 else (255, 255, 255)
        cv2.putText(frame, f"{Action(i).name.lower()}: {(action_prob*100):.2f} %", (110, 50 + i*20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, label_color, 1)

In [ ]:
class ExponentialSmoothing:
    def __init__(self, alpha):
        self.alpha = alpha
        self.smoothed_probabilities = None

    def update(self, probabilities):
        if self.smoothed_probabilities is None:
            self.smoothed_probabilities = probabilities
        else:
            self.smoothed_probabilities = self.alpha*probabilities + (1 - self.alpha)*self.smoothed_probabilities
        return self.smoothed_probabilities

In [ ]:
# Load models (TorchScript)

# Action recognition
model_ar = torch.jit.load(os.path.join("weights", "model_ar_simple_lstm.pt"))
model_ar.eval()

# Action prediction
model_ap = torch.jit.load(os.path.join("weights", "model_ap.pt"))
model_ap.eval()

In [ ]:
def recognize_action(data_buffer, smoother=None):
    X = torch.tensor(np.array(data_buffer), dtype=torch.float32).to(device)
    X = torch.unsqueeze(X, axis=0)
    with torch.no_grad():
        probabilities = F.softmax(model_ar(X)[0], dim=0)
        if smoother is not None:
            return smoother.update(probabilities)
        else:
            return probabilities

In [ ]:
def register_action_in_sequence(action_index, currently_in_hand_tracked_objects, actions_list, right_hand_objects_list, left_hand_objects_list):
    right_hand_object_index = currently_in_hand_tracked_objects[0].yolo_object.label_id if currently_in_hand_tracked_objects[0] is not None else len(OBJECT_NAMES)
    left_hand_object_index = currently_in_hand_tracked_objects[1].yolo_object.label_id if currently_in_hand_tracked_objects[1] is not None else len(OBJECT_NAMES)
    print("Registering new action:", action_index, right_hand_object_index) # left_hand_object_index
    actions_list.append(action_index)
    right_hand_objects_list.append(right_hand_object_index)
    left_hand_objects_list.append(left_hand_object_index)

def one_hot_encode_class(class_index, num_classes):
    one_hot_vector = np.zeros(num_classes, dtype=int)
    one_hot_vector[class_index] = 1
    return one_hot_vector

def predict_next_action(one_hot_sequence_data):
    """
    Predict the next most probable action based on provided one-hot sequence data.
    Returns the probabilities for predicted actions and objects.
    """
    X = torch.tensor(np.array(one_hot_sequence_data), dtype=torch.float32).to(device)
    X = torch.unsqueeze(X, axis=0)
    with torch.no_grad():
        outputs1, outputs2 = model_ap(X)
        prob1 = F.softmax(outputs1, dim=1)[0]
        prob2 = F.softmax(outputs2, dim=1)[0]
        return prob1, prob2

def search_for_future_action(sequence_data, action_to_find, force_object=False):
    """
    Uses action prediction to search for a specified future action.
    
    Parameters:
        sequence_data: numpy.ndarray, shape (sequence_length, 2)
            Sequence data in pairs (tuple): (action_index, object_index).
        action_to_find: Action
            Action to search for in the future.
        force_object: bool, default False
            If True, if the object found for the specified action is none,
            the second most probable object will be returned.
    
    Returns:
        prediction_pairs: list
            List of predicted pairs from the next step to the action to find.
        conf: tuple (action_confidence, object_confidence)
            Confidence in the action to be found, calculated as the conditional probability of 
            the previous action confidences in the prediction_pairs (multiplied confidences),
            ignoring idle actions, and the confidence of the relative predicted object.
        object_forced: bool
            Whether the object found for the specified action was forced.
            It always returns False if force_object is False.
    """
    one_hot_sequence_data = deque(maxlen=PREDICTION_WINDOW_SIZE)
    for action_pair in sequence_data:
        one_hot_sequence_data.append(np.concatenate((one_hot_encode_class(action_pair[0], num_classes_actions), one_hot_encode_class(action_pair[1], num_classes_objects)), axis=0))

    prediction_pairs = []
    confidences = []
    object_forced = False
    while True:
        prob1, prob2 = predict_next_action(one_hot_sequence_data)
        action_prediction_index = torch.argmax(prob1).item()
        object_prediction_indices = torch.sort(prob2, descending=True).indices.tolist()
        
        if Action(action_prediction_index) == action_to_find: # Action found
            if force_object and object_prediction_indices[0] == len(OBJECT_NAMES): # Object forced
                prediction_pairs.append((action_prediction_index, object_prediction_indices[1]))
                confidences.append((prob1[action_prediction_index].item(), prob2[object_prediction_indices[1]].item()))
                object_forced = True
            else: # Object not forced
                prediction_pairs.append((action_prediction_index, object_prediction_indices[0]))
                confidences.append((prob1[action_prediction_index].item(), prob2[object_prediction_indices[0]].item()))
            break
        else: # Add predicted row to sequence and predict next
            action_pair = (action_prediction_index, object_prediction_indices[0])
            prediction_pairs.append(action_pair)
            confidences.append((prob1[action_pair[0]].item(), prob2[action_pair[1]].item()))
            one_hot_sequence_data.append(np.concatenate((one_hot_encode_class(action_pair[0], num_classes_actions), one_hot_encode_class(action_pair[1], num_classes_objects)), axis=0))

    action_conditional_probability = 1.0
    for i in range(len(confidences)):
        if prediction_pairs[i][0] != Action.IDLE.value:
            action_conditional_probability *= confidences[i][0]

    return prediction_pairs, tuple((action_conditional_probability, confidences[-1][1])), object_forced

In [ ]:
def pick_and_deliver_object(hand_helper, object_tracker, camera, robot_controller, actions_list, right_hand_objects_list, left_hand_objects_list, execution_queue):
    robot_controller.is_moving = True
    while execution_queue:
        pick_action_pair = execution_queue.pop(0)
        to_pick_object_label_id = pick_action_pair[1]
        to_pick_tracked_object = None
        for tracked_object in object_tracker.tracked_objects:
            if tracked_object.yolo_object.label_id == to_pick_object_label_id and tracked_object.is_visible and not tracked_object.is_moving \
               and not (object_tracker.right_hand_tracked_object and object_tracker.right_hand_tracked_object.tracker_id == tracked_object.tracker_id) \
               and not (object_tracker.left_hand_tracked_object and object_tracker.left_hand_tracked_object.tracker_id == tracked_object.tracker_id):
                cx, cy = tracked_object.yolo_object.get_center()
                depth = camera.get_depth(cx, cy)
                P_cam = np.array(camera.deproject_pixel_to_point(cx, cy, depth) + [1])
                P_rob = robot_controller.cam_to_robot_transform(P_cam)
                x, y = P_rob[0], P_rob[1]
                z_approach = 100.0
                z = 25.0
                if robot_controller.check_workspace(x, y, z_approach) and robot_controller.check_workspace(x, y, z):
                    to_pick_tracked_object = tracked_object
                    break
        # Picking
        if to_pick_tracked_object is not None:
            z_shift = z - z_approach
            approaching_speed = 130 if robot_controller.is_at_home_pos else 80
            robot_controller.move_cartesian(x=x, y=y, z=z_approach, speed=approaching_speed)
            robot_controller.shift_cartesian_rel_base(z_shift=z_shift, speed=80)
            robot_controller.is_delivering = True
            robot_controller.close_gripper()
            robot_controller.shift_cartesian_rel_base(z_shift=-z_shift, speed=80)
            # Delivering
            delivered = False
            while not delivered and robot_controller.is_connected:
                right_hand_center = hand_helper.get_hand_centers()[0]
                cx, cy = right_hand_center[0], right_hand_center[1]
                if cx > 0 and cy > 0:
                    depth = camera.get_depth(cx, cy)
                    P_cam = np.array(camera.deproject_pixel_to_point(cx, cy, depth) + [1])
                    P_rob = robot_controller.cam_to_robot_transform(P_cam)
                    x_hand, y_hand, z_hand = P_rob[0], P_rob[1], P_rob[2]
                    z_hand += 50
                    current_robot_pose = robot_controller.get_current_pose_cartesian()
                    distance = (x_hand - current_robot_pose[0])**2 + (y_hand - current_robot_pose[1])**2 + (z_hand - current_robot_pose[2])**2
                    distance_threshold = 1600
                    if distance <= distance_threshold:
                        robot_controller.open_gripper()
                        delivered = True
                        robot_controller.is_delivering = False
                        object_tracker.force_object_in_hand(0, to_pick_tracked_object)
                        register_action_in_sequence(Action.PICK.value, (to_pick_tracked_object, None), actions_list, right_hand_objects_list, left_hand_objects_list)
                        sequence_data = list(zip(actions_list, right_hand_objects_list))
                        if len(sequence_data) >= PREDICTION_WINDOW_SIZE:
                            prediction_pairs, conf, object_forced = search_for_future_action(sequence_data[-PREDICTION_WINDOW_SIZE:], Action.PICK, force_object=True)
                            print("Prediction", prediction_pairs, conf, object_forced)
                            # Action anticipation logic
                            pick_action_pair = prediction_pairs[-1]
                            if pick_action_pair[0] == Action.PICK.value: # Redundant check
                                if conf[0] > 0.8 and conf[1] > 0.6:
                                    execution_queue.append(pick_action_pair)
                                else:
                                    print(f"Action {pick_action_pair} not executed for not enough confidence: {conf}")
                    elif robot_controller.check_workspace(x_hand, y_hand, z_hand):
                        robot_controller.move_cartesian(x=x_hand, y=y_hand, z=z_hand, speed=80)
                time.sleep(0.3)

        else:
            print(f"No object with label {to_pick_object_label_id} found in workspace")

        if not execution_queue and not robot_controller.is_at_home_pos:
            robot_controller.move_to_home_pose()
    
    robot_controller.is_moving = False

In [ ]:
camera = RealSenseCamera(image_width=IMAGE_WIDTH, image_height=IMAGE_HEIGHT)
camera.connect()

robot_controller = RobotController()
robot_controller.connect()
robot_controller.move_to_home_pose()
robot_controller.open_gripper()

prev_frame_time = 0
new_frame_time = 0

first_frame_acquisition_time = 0
frame_index = 0

data_buffer = deque(maxlen=sequence_length)
action_recognition_probabilities = None

hand_helper = HandHelper(image_width=IMAGE_WIDTH, image_height=IMAGE_HEIGHT)
object_tracker = ObjectTracker(image_width=IMAGE_WIDTH, image_height=IMAGE_HEIGHT)

smoother = ExponentialSmoothing(alpha=0.4)

current_top_action_index = -1
actions_list = []
right_hand_objects_list = []
left_hand_objects_list = []
execution_queue = []

try:
    with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while True:
            ret = camera.acquire_frame()
            if not ret:
                break
            
            color_image = camera.color_image

            hands_results = process_landmarks(color_image, hands)
            objects_results = yolo_model(color_image, verbose=False)[0]

            new_frame_time = time.time()
            fps = 1.0/(new_frame_time - prev_frame_time)
            prev_frame_time = new_frame_time
            cv2.putText(color_image, f"{fps:.2f}", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1)

            right_hand_landmarks = extract_hand_landmarks(0, hands_results.multi_hand_landmarks, hands_results.multi_handedness)
            left_hand_landmarks = extract_hand_landmarks(1, hands_results.multi_hand_landmarks, hands_results.multi_handedness)
            hand_helper.register_hands_landmarks(right_hand_landmarks, left_hand_landmarks)
            tips_midpoints = hand_helper.get_tips_midpoints()
            hand_centers = hand_helper.get_hand_centers()

            draw_landmarks(color_image, right_hand_landmarks, side=0, connections=HAND_CONNECTIONS)
            draw_landmarks(color_image, left_hand_landmarks, side=1, connections=HAND_CONNECTIONS)
            # cv2.circle(color_image, tips_midpoints[0], 5, (0, 255, 0), -1)
            # cv2.circle(color_image, tips_midpoints[1], 5, (0, 255, 0), -1)
            # cv2.circle(color_image, hand_centers[0], 5, (0, 255, 255), -1)
            # cv2.circle(color_image, hand_centers[1], 5, (0, 255, 255), -1)

            seen_yolo_objects = []
            for object_result in objects_results.boxes:
                if object_result.conf.item() > 0.75:
                    seen_yolo_objects.append(YoloObject.from_yolo_box_result(object_result))

            object_tracker.register_seen_objects(seen_yolo_objects, tips_midpoints)
            object_tracker.increment_frame_index()
            
            for i, in_hand_tracked_object in enumerate((object_tracker.right_hand_tracked_object, object_tracker.left_hand_tracked_object)):
                if in_hand_tracked_object is None:
                    continue
                hand_color = (0, 0, 255) if i == 0 else (255, 0, 0)
                yolo_object = in_hand_tracked_object.yolo_object
                cv2.putText(color_image, f"{OBJECT_NAMES[yolo_object.label_id]}, visible: {in_hand_tracked_object.is_visible}", (10, 140 + i*25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, hand_color, 1)
                if in_hand_tracked_object.is_visible:
                    cv2.rectangle(color_image, (yolo_object.x1, yolo_object.y1), (yolo_object.x2, yolo_object.y2), hand_color, 2)
                    cv2.putText(color_image, f"{OBJECT_NAMES[yolo_object.label_id]}, conf: {yolo_object.conf:.2f}",
                                (yolo_object.x1, yolo_object.y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, hand_color, 1)
            
            # Action recognition logic
            elapsed_time = time.time() - first_frame_acquisition_time - frame_index*frame_interval
            if elapsed_time > frame_interval:
                hands_landmarks = right_hand_landmarks + left_hand_landmarks 
                frame_landmarks_data = np.concatenate([lm.get_np_array() for lm in hands_landmarks])
                """ # Also include objects in model input
                frame_objects_data = []
                for i, in_hand_tracked_object in enumerate((object_tracker.right_hand_tracked_object, object_tracker.left_hand_tracked_object)):
                    if in_hand_tracked_object is None:
                        continue
                    yolo_object = in_hand_tracked_object.yolo_object
                    frame_objects_data.append(np.array((yolo_object.label_id, i, yolo_object.x1, yolo_object.y1, yolo_object.x2, yolo_object.y2))) """
                selected_frame_data = get_selected_frame_data(frame_landmarks_data, None)
                data_buffer.append(selected_frame_data)
                if len(data_buffer) == sequence_length:
                    action_recognition_probabilities = recognize_action(data_buffer, smoother=smoother)
                    best_action_index = torch.argmax(action_recognition_probabilities).item()
                    # Action prediction logic (looking for next PICK action)
                    if action_recognition_probabilities[best_action_index] > 0.9 \
                       and not (actions_list and actions_list[-1] == best_action_index) \
                       and current_top_action_index != best_action_index:
                        current_top_action_index = best_action_index
                        if not robot_controller.is_delivering: # Not registering in sequence if the robot is delivering
                            register_action_in_sequence(best_action_index, (object_tracker.right_hand_tracked_object, object_tracker.left_hand_tracked_object), actions_list, right_hand_objects_list, left_hand_objects_list)
                            if not robot_controller.is_moving or robot_controller.is_at_home_pos: # Not predicting if the robot is moving
                                sequence_data = list(zip(actions_list, right_hand_objects_list))
                                if len(sequence_data) >= PREDICTION_WINDOW_SIZE:
                                    prediction_pairs, conf, object_forced = search_for_future_action(sequence_data[-PREDICTION_WINDOW_SIZE:], Action.PICK, force_object=True)
                                    print("Prediction", prediction_pairs, conf, object_forced)
                                    # Action anticipation logic
                                    pick_action_pair = prediction_pairs[-1]
                                    if pick_action_pair[0] == Action.PICK.value: # Redundant check
                                        if conf[0] > 0.8 and conf[1] > 0.6:
                                            execution_queue.append(pick_action_pair)
                                            if not robot_controller.is_moving:
                                                threading.Thread(target=pick_and_deliver_object, args=(hand_helper, object_tracker, camera, robot_controller, actions_list, right_hand_objects_list, left_hand_objects_list, execution_queue)).start()
                                        else:
                                            print(f"Action {pick_action_pair} not executed for not enough confidence: {conf}")

                if frame_index == 0:
                    first_frame_acquisition_time = time.time()
                    
                frame_index += 1

            if action_recognition_probabilities is not None:
                show_probability_bars(color_image, action_recognition_probabilities)
            
            cv2.imshow("RealSense", color_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

finally:
    cv2.destroyAllWindows()
    camera.disconnect()
    robot_controller.disconnect()

In [ ]:
print(actions_list, len(actions_list))
print(right_hand_objects_list, len(right_hand_objects_list))
print(left_hand_objects_list, len(left_hand_objects_list))
for i in range(len(actions_list)):
    action_name = Action(actions_list[i]).name.lower()
    right_hand_object_name = OBJECT_NAMES[right_hand_objects_list[i]] if right_hand_objects_list[i] < len(OBJECT_NAMES) else "none"
    # left_hand_object_name = OBJECT_NAMES[left_hand_objects_list[i]] if left_hand_objects_list[i] < len(OBJECT_NAMES) else "none"
    print(f"Action: {action_name}, Right: {right_hand_object_name}") # , Left: {left_hand_object_name}")

In [ ]:
def save_observation(actions_list, right_hand_objects_list, left_hand_objects_list): # Now just considering right hand
    OBSERVATIONS_FOLDER = "observations"

    observation_number = 0
    if os.path.exists(OBSERVATIONS_FOLDER):
        observation_names = os.listdir(OBSERVATIONS_FOLDER)
        if observation_names:
            observation_numbers = []
            for observation_name in observation_names:
                observation_numbers.append(int(observation_name.split('.')[0].split('_')[1]))
            observation_number = sorted(observation_numbers, reverse=True)[0] + 1

    observation_path = os.path.join(OBSERVATIONS_FOLDER, f"observation_{observation_number}.csv")

    rows = []
    for i in range(len(actions_list)):
        row_dict = {"action": actions_list[i], "right_hand_object": right_hand_objects_list[i]}
        rows.append(row_dict)
        
    pd.DataFrame(rows).to_csv(observation_path, encoding="utf-8", index=False)
    print(f"Saved new observation (just right hand) at {observation_path}")

In [ ]:
save_observation(actions_list, right_hand_objects_list, left_hand_objects_list)